
# Occupany Grid Mapping

This problem of estimating the full state of a 100 x 100 world can be treated as a Dynamic Occupancy Grid Mapping. The assumption that data is noise-free makes it possible to use staright forward startegies:
*  Last Best ValueHit/Miss counter 

For a noiseless setting, the sensor model will look like this:
>$P(Z| Occlusion) = 1$             ,sensor only gives a measurement when an obstacle is detected

>$P(Z| !Occlusion) = 0$            ,no measurments for empty space


For a noisy setting, a sensor model might look like this:

>$P(Z| Occlusion) = 0.8$            ,80% of the times an obstacle is detected, sensor gives a reading

>$P(Z| !Occlusion) = 0.3$           ,30% of the times in ama epmty space, sensor gives false readings

In [ ]:
import numpy as np

In [3]:
def findPosterior(P_Occ, P_Z_Occ, P_Z_NOcc):
    P_Z = (P_Z_Occ*P_Occ) + (P_Z_NOcc*(1- P_Occ))
    P_Occ_Z = (P_Z_Occ*P_Occ) / P_Z
    return P_Occ_Z

In [10]:
print findPosterior(0.3, 0.2, 1)

0.0789473684211


## Observation Matrix, Bz
3 Matrices which describe the observation model. This is pre-decided. Noise can be added here.

$\mathbf{B_z(h)} = \left[\begin{array}{rc}
p(h | Occ) & 0\\
0 & p(h | Empty) \\
\end{array}\right]
$

$\mathbf{h \in (hit, miss, no\_obs)}$

